In [41]:
# Import Packages
import pandas as pd, numpy as np
import os, sys, glob, re
from pathlib import Path
from itertools import compress
import string

In [42]:
inputfilepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\01_download_cc\01.2_xls_combined\multiple_data_pulls_combined\20010101-20220617\xls_compiled_20010101-20220617.csv")
xlscombined_withcleanfirmnames_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.2_firm_names_to_match\multiple_data_pulls_combined\20010101-20220617\xlscombined_with_cleanfirmnames_20010101-20220617.xlsx")
cleanfirmnamestomatch_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.2_firm_names_to_match\multiple_data_pulls_combined\20010101-20220617\cleanfirmnames_to_match_20010101-20220617.xlsx")

In [43]:
df = pd.read_csv(inputfilepath, low_memory=False)
df.head(1)

,index,PPV,TOC,Title,Subtitle,Date,Pages,Price,Contributor,Analyst,Ratings,Language,Report #,Collection,filestem
0,NaN,N,Y,FUEL SYSTEMS SOLUTIONS INC,FSYS.OQ - Event Transcript of Fuel Systems Sol...,11/08/12,17.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21044925.0,INV,NaN


In [44]:
# Remove empty rows
print("Number of rows of df:", df.shape[0])
print("Number of empty rows of df:", df['Title'].isna().sum())
df = df[~df['Title'].isna()]
print("Number of rows of df, after removing empty rows:", df.shape[0])

Number of rows of df: 646310
Number of empty rows of df: 2
Number of rows of df, after removing empty rows: 646308


In [86]:
def get_clean_firm_name(title):
    # Convert to upper case
    clean_firm_name = title.upper()
    
    # Use the same abbreviated firm suffix (e.g. incorporated to inc, corporation to corp, limited to ltd)
    clean_firm_name = re.sub('INCORPORATED', 'INC', clean_firm_name)
    clean_firm_name = re.sub('CORPORATION', 'CORP', clean_firm_name)
    clean_firm_name = re.sub('LIMITED', 'LTD', clean_firm_name)
    clean_firm_name = re.sub('COMPANY', 'CO', clean_firm_name)
    clean_firm_name = re.sub('AKTIENGESELLSCHAFT', 'AG', clean_firm_name)
    clean_firm_name = re.sub('INTERNATIONAL', 'INTL', clean_firm_name)
    
    # We want to convert titles like:
    # "- EVENT TRANSCRIPT OF ADELAIDE CAPITAL MARKETS CONFERENCE CALL" to "ADELAIDE CAPITAL MARKETS", and
    # "- EVENT TRANSCRIPT OF CAIXA GERAL DE DEPOSITOS SA CONFERENCE C" to "CAIXA GERAL DE DEPOSITOS SA"
    
    # Note: more removals after including older conference calls
    
    # Remove everything before "EVENT TRANSCRIPT OF" or "EVENT BRIEF OF" 
    clean_firm_name = re.sub('^.*EVENT TRANSCRIPT OF ', '', clean_firm_name)
    clean_firm_name = re.sub('^.*EVENT BRIEF OF ', '', clean_firm_name)
    
    # Remove formats like "... (<firm ticker>) - ..."
    clean_firm_name = re.sub('\(\w+\) - .+$', '', clean_firm_name)
    
    # Remove formats like "- PRELIMINARY TRANSCRIPT ..."
    clean_firm_name = re.sub('- *PRELIMINARY TRANSCRIPT.*$', '', clean_firm_name)
    clean_firm_name = re.sub('- *FINAL TRANSCRIPT.*$', '', clean_firm_name)
    clean_firm_name = re.sub('- *EDITED BRIEF.*$', '', clean_firm_name)
    clean_firm_name = re.sub('- *EDITED TRANSCRIPT.*$', '', clean_firm_name)
    clean_firm_name = re.sub('- *PRELIMINARY.*$', '', clean_firm_name)
    
    # Remove everything after "CONFERENCE"
    clean_firm_name = re.sub('- .+ EARNINGS .*$', '', clean_firm_name)
    clean_firm_name = re.sub('EARNINGS CONFERENCE.*$', '', clean_firm_name)
    clean_firm_name = re.sub(' CONFERENCE.*$', '', clean_firm_name)
    clean_firm_name = re.sub('FINANCIAL RELEASE C.*$', '', clean_firm_name)
    
    # Remove everything after "Q\d \d\d\d\d", e.g. "Q1 2001"
    # Also "\dQ \d\d\d\d", e.g. "1Q 2001".
    # Used * instead of + because the title may end with Q1 2001, e.g. "<firm> Q1 2001"
    clean_firm_name = re.sub(' Q\d \d{4}.*$', '', clean_firm_name)
    clean_firm_name = re.sub(' \dQ \d{4}.*$', '', clean_firm_name)
    
    # Remove punctuation
    clean_firm_name = clean_firm_name.translate(str.maketrans('', '', string.punctuation))

    # Remove extra spaces
    clean_firm_name = clean_firm_name.strip()
    clean_firm_name = re.sub(' +', ' ', clean_firm_name)
    
    return clean_firm_name

In [88]:
# Clean titles to get clean firm names
df['clean_firm_name'] = df['Title'].apply(get_clean_firm_name)
print("Number of rows of df:", df.shape[0])
df.head(5)

Number of rows of df: 646308


,index,PPV,TOC,Title,Subtitle,Date,Pages,Price,Contributor,Analyst,Ratings,Language,Report #,Collection,filestem,clean_firm_name
0,NaN,N,Y,FUEL SYSTEMS SOLUTIONS INC,FSYS.OQ - Event Transcript of Fuel Systems Sol...,11/08/12,17.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21044925.0,INV,NaN,FUEL SYSTEMS SOLUTIONS INC
1,NaN,N,Y,NN INC,NNBR.OQ - Event Transcript of NN Inc conferenc...,11/08/12,18.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21045694.0,INV,NaN,NN INC
2,NaN,N,Y,COPANO ENERGY LLC,CPNO.OQ - Event Transcript of Copano Energy LL...,11/08/12,11.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21045712.0,INV,NaN,COPANO ENERGY LLC
3,NaN,N,Y,SODEXO,EXHO.PA - Event Transcript of Sodexo SA confer...,11/08/12,23.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21045002.0,INV,NaN,SODEXO
4,NaN,N,Y,VULCAN MATERIALS CO.,VMC.N - Event Transcript of Vulcan Materials C...,11/08/12,21.0,Subscription,THOMSON REUTERS STREETEVENTS,ANON,NaN,English,21045791.0,INV,NaN,VULCAN MATERIALS CO


In [89]:
# Get unique clean firm names. These will be the firms we want to match against compustat/hassan.
clean_firm_names = pd.DataFrame(df['clean_firm_name'].value_counts())
clean_firm_names.reset_index(inplace=True)
clean_firm_names = clean_firm_names.rename(columns = {'clean_firm_name': 'count', 'index':'clean_firm_name'})
print("Number of unique clean firm names:", clean_firm_names.shape[0])
clean_firm_names

Number of unique clean firm names: 21604


,clean_firm_name,count
0,FORD MOTOR CO,657
1,SALESFORCE COM INC,523
2,ALNYLAM PHARMACEUTICALS INC,484
3,AMGEN INC,418
4,ELI LILLY CO,399
...,...,...
21599,ALLIS CHALMERS ENERGY INC ALLIS CHALMERS ENERG...,1
21600,MACDONALD DETTWILER AND ASSOCIATE,1
21601,PILLOWTEX CORP,1
21602,NATIONSHEALTH INC,1


Last result: 21604 unique clean firm names

In [90]:
# Save output as .xlsx files
df.to_excel(xlscombined_withcleanfirmnames_filepath, index=False)
print("Saved xls combined with clean firm names to:", xlscombined_withcleanfirmnames_filepath)

clean_firm_names.to_excel(cleanfirmnamestomatch_filepath, index=False)
print("\nSaved clean firm names to match to:", cleanfirmnamestomatch_filepath)

Saved xls combined with clean firm names to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.2_firm_names_to_match\multiple_data_pulls_combined\20010101-20220617\xlscombined_with_cleanfirmnames_20010101-20220617.xlsx

Saved clean firm names to match to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.2_firm_names_to_match\multiple_data_pulls_combined\20010101-20220617\cleanfirmnames_to_match_20010101-20220617.xlsx
